# Error in 3D

In this notebook we load the points OpenPTV creates errors in 3D from 
Robin generated 2D and 3D points, see also proPTV_LineOfSight/generate_error.py

and markers_c0.txt, ...


In [15]:
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go

import pathlib, os
working_path = pathlib.Path.cwd()

In [16]:
class Parameter:
    cams = [0,1,2,3]
    Vmin = [0,0,0]
    Vmax = [300,300,300]
    N1, N2 = 361, 5

In [17]:
params = Parameter()

markers = [np.loadtxt('../proPTV_LineOfSight/markers_c'+str(cam)+'.txt') for cam in params.cams]
XYZ = markers[0][:,2:]
xy = markers[0][:,:2]
ID = np.argwhere((XYZ[:,0]>-1))[:,0]

In [18]:
xyz = pd.DataFrame(XYZ, columns=['x','y','z'])
xyz['id'] = ID
px.scatter_3d(x=xyz['x'], y=xyz['y'], z=xyz['z'], color=xyz['id']).show()

In [19]:
# First, let's calibrate roughly the cameras

In [20]:

ref_pts = XYZ[[0,721,1409,1462],:]
xyz = pd.DataFrame(ref_pts, columns=['x','y','z'])
xyz['id'] = ID[[0,721,1409,1462]]
px.scatter_3d(x=xyz['x'], y=xyz['y'], z=xyz['z'], color=xyz['id']).show()


In [21]:

from openptv_python.calibration import Calibration
from openptv_python.parameters import ControlPar, VolumePar

cal = Calibration().from_file(working_path / "calibration" / "cam1.tif.ori", None)
cpar = ControlPar().from_file(working_path / "parameters" / "ptv.par")
vpar = VolumePar().from_file(working_path / "parameters" / "criteria.par")


No addpar file found. Using default values for radial distortion


In [22]:
print(f"{cal.ext_par = }")

cal.ext_par = rec.array((351.70698916, -20.06356359, 1144.5560229, 0.18701021, 0.20874593, -0.03370383, [[ 0.97773598,  0.03296593,  0.20723322], [ 0.00539743,  0.98330481, -0.18188599], [-0.20976946,  0.17895501,  0.96123456]]),
          dtype=[('x0', '<f8'), ('y0', '<f8'), ('z0', '<f8'), ('omega', '<f8'), ('phi', '<f8'), ('kappa', '<f8'), ('dm', '<f8', (3, 3))])


In [23]:
cal.update_rotation_matrix()

In [24]:

from openptv_python.imgcoord import image_coordinates
from openptv_python.trafo import arr_metric_to_pixel
from openptv_python.orientation import external_calibration



In [25]:
four_points = xy[[0,721,1409,1462],:]
print(f"{four_points = }")

four_points = array([[ 315., 1996.],
       [2133.,  235.],
       [ 892.,  561.],
       [2215., 2018.]])


In [26]:
external_calibration(cal, ref_pts, four_points, cpar)
print(f"{cal.ext_par = }")

cal.ext_par = rec.array((358.52144247, -13.04339097, 1144.95187596, 0.17998199, 0.21458089, -0.03178908, [[ 0.97657208,  0.03105479,  0.21293794], [ 0.00682883,  0.98456139, -0.17490634], [-0.21508216,  0.17226277,  0.9612831 ]]),
          dtype=[('x0', '<f8'), ('y0', '<f8'), ('z0', '<f8'), ('omega', '<f8'), ('phi', '<f8'), ('kappa', '<f8'), ('dm', '<f8', (3, 3))])


In [27]:
targets = arr_metric_to_pixel(
    image_coordinates(ref_pts, cal, cpar.mm),
cpar,
)
four_points - targets

array([[-4.8897183 ,  4.85307797],
       [ 6.36175491, -6.06834184],
       [ 6.55511064,  9.11383901],
       [-7.75188026, -7.62917658]])

In [28]:
def error_function(x, cal, XYZ, xy, cpar):
    
    # cal.ext_par['x0'], cal.ext_par['y0'], cal.ext_par['z0'] = x[:3]
    # cal.ext_par['omega'], cal.ext_par['phi'], cal.ext_par['kappa'] = x[3:]
    # cal.update_rotation_matrix()
    cal.set_pos(x[:3])
    cal.set_angles(x[3:])
    
    targets = arr_metric_to_pixel(
        image_coordinates(XYZ, cal, cpar.mm),
    cpar,
    )
    
    return np.sum(np.abs(xy - targets))

In [29]:
x0 = cal.ext_par.tolist()[:6]
error_function(x0, cal, XYZ, xy, cpar)

17522.376198244092

In [30]:
from scipy.optimize import minimize

x0 = cal.ext_par.tolist()[:6]

sol = minimize(error_function, x0, args=(cal, XYZ, xy, cpar), method='Nelder-Mead')

In [ ]:
cal.set_pos(sol.x[:3])
cal.set_angles(sol.x[3:])

targets = arr_metric_to_pixel(
    image_coordinates(ref_pts, cal, cpar.mm),
cpar,
)
four_points - targets


array([[-11.36709035,   7.25184957],
       [  2.08925134,  -4.75418336],
       [  5.36335193,   5.65986942],
       [-11.09295142,  -8.46084412]])

In [39]:
targets = arr_metric_to_pixel(
    image_coordinates(XYZ, cal, cpar.mm),
cpar,
)

import plotly.figure_factory as ff
# px.scatter(x=xy[:,0], y=xy[:,1], color=ID).show()
fig = ff.create_quiver(x=xy[:,0], y=xy[:,1], u=targets[:,0]-xy[:,0], v=targets[:,1]-xy[:,1], scale=10)
fig.show()


In [32]:
from openptv_python.parameters import OrientPar
from openptv_python.orientation import full_calibration

orient_par = OrientPar().from_file(working_path / "parameters" / "orient.par")

orient_par.ccflag=0
orient_par.xhflag=1
orient_par.yhflag=1
orient_par.k1flag=0
orient_par.k2flag=0
orient_par.k3flag=0
orient_par.scxflag=0
orient_par.sheflag=0


_, _, _ = full_calibration(
    cal,
    XYZ,
    xy,
    cpar,
    orient_par
    )

print(f"{cal.ext_par = }")


Residuals = None


ValueError: Orientation iteration failed, need better setup.

In [ ]:

np.testing.assert_array_almost_equal(
    self.cal.get_angles(), self.orig_cal.get_angles(), decimal=4
)
np.testing.assert_array_almost_equal(
    self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
)

print(f"{self.cal.get_pos()}")
print(f"{self.cal.get_angles()}")
print(f"{self.cal.added_par}")

# Perturb the calibration object, then compore result to original.
self.cal.set_pos(self.cal.get_pos() + np.r_[1.0, -1.0, 1.0])
self.cal.set_angles(self.cal.get_angles() + np.r_[-0.1, 0.1, -0.1])

self.orient_par.ccflag=1
self.orient_par.xhflag=1
self.orient_par.yhflag=1
print(f"Calibrating with the following flags: {self.orient_par}")

_, _, _ = full_calibration(
    self.cal,
    ref_pts,
    targets,
    self.control,
    self.orient_par
    )

np.testing.assert_array_almost_equal(
    self.cal.get_angles(), self.orig_cal.get_angles(), decimal=4
)
np.testing.assert_array_almost_equal(
    self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
)

print(f"{self.cal.get_pos()}")
print(f"{self.cal.get_angles()}")
print(f"{self.cal.added_par}")

# Perturb the calibration object, then compore result to original.
# self.cal.set_pos(self.cal.get_pos() + np.r_[1.0, -1.0, 1.0])
# self.cal.set_angles(self.cal.get_angles() + np.r_[-0.1, 0.1, -0.1])

self.orient_par.ccflag=0
self.orient_par.xhflag=0
self.orient_par.yhflag=0
self.orient_par.k1flag=0
self.orient_par.k2flag=0
self.orient_par.k3flag=0
self.orient_par.scxflag=0
self.orient_par.sheflag=0
print(f"Calibrating with the following flags: {self.orient_par}")

_, _, _ = full_calibration(
    self.cal,
    ref_pts,
    targets,
    self.control,
    self.orient_par
    )

np.testing.assert_array_almost_equal(
    self.cal.get_angles(), self.orig_cal.get_angles(), decimal=4
)
np.testing.assert_array_almost_equal(
    self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
)
print(f"{self.cal.get_pos()}")
print(f"{self.cal.get_angles()}")
print(f"{self.cal.added_par}")

self.orient_par.ccflag=0
self.orient_par.xhflag=0
self.orient_par.yhflag=0
self.orient_par.k1flag=0
self.orient_par.k2flag=0
self.orient_par.k3flag=0
self.orient_par.scxflag=1
self.orient_par.sheflag=0
print(f"Calibrating with the following flags: {self.orient_par}")

_, _, _ = full_calibration(
    self.cal,
    ref_pts,
    targets,
    self.control,
    self.orient_par
    )

np.testing.assert_array_almost_equal(
    self.cal.get_angles(), self.orig_cal.get_angles(), decimal=4
)
np.testing.assert_array_almost_equal(
    self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
)
print(f"{self.cal.get_pos()}")
print(f"{self.cal.get_angles()}")
print(f"{self.cal.added_par}")

self.orient_par.ccflag=0
self.orient_par.xhflag=0
self.orient_par.yhflag=0
self.orient_par.k1flag=0
self.orient_par.k2flag=0
self.orient_par.k3flag=1
self.orient_par.scxflag=0
self.orient_par.sheflag=0
print(f"Calibrating with the following flags: {self.orient_par}")

_, _, _ = full_calibration(
    self.cal,
    ref_pts,
    targets,
    self.control,
    self.orient_par
    )

np.testing.assert_array_almost_equal(
    self.cal.get_angles(), self.orig_cal.get_angles(), decimal=4
)
np.testing.assert_array_almost_equal(
    self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
)
print(f"{self.cal.get_pos()}")
print(f"{self.cal.get_angles()}")
print(f"{self.cal.added_par}")

self.orient_par.ccflag=0
self.orient_par.xhflag=0
self.orient_par.yhflag=0
self.orient_par.k1flag=0
self.orient_par.k2flag=1
self.orient_par.k3flag=0
self.orient_par.scxflag=0
self.orient_par.sheflag=0
print(f"Calibrating with the following flags: {self.orient_par}")

_, _, _ = full_calibration(
    self.cal,
    ref_pts,
    targets,
    self.control,
    self.orient_par
    )

np.testing.assert_array_almost_equal(
    self.cal.get_angles(), self.orig_cal.get_angles(), decimal=4
)
np.testing.assert_array_almost_equal(
    self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
)
print(f"{self.cal.get_pos()}")
print(f"{self.cal.get_angles()}")
print(f"{self.cal.added_par}")

self.orient_par.ccflag=0
self.orient_par.xhflag=0
self.orient_par.yhflag=0
self.orient_par.k1flag=1
self.orient_par.k2flag=0
self.orient_par.k3flag=0
self.orient_par.scxflag=0
self.orient_par.sheflag=0
print(f"Calibrating with the following flags: {self.orient_par}")

_, _, _ = full_calibration(
    self.cal,
    ref_pts,
    targets,
    self.control,
    self.orient_par
    )

np.testing.assert_array_almost_equal(
    self.cal.get_angles(), self.orig_cal.get_angles(), decimal=4
)
np.testing.assert_array_almost_equal(
    self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
)
print(f"{self.cal.get_pos()}")
print(f"{self.cal.get_angles()}")
print(f"{self.cal.added_par}")

self.orient_par.ccflag=0
self.orient_par.xhflag=0
self.orient_par.yhflag=0
self.orient_par.k1flag=0
self.orient_par.k2flag=0
self.orient_par.k3flag=1
self.orient_par.scxflag=0
self.orient_par.sheflag=0
self.orient_par.p1flag=1
self.orient_par.p2flag=0

print(f"Calibrating with the following flags: {self.orient_par}")

_, _, _ = full_calibration(
    self.cal,
    ref_pts,
    targets,
    self.control,
    self.orient_par
    )

np.testing.assert_array_almost_equal(
    self.cal.get_angles(), self.orig_cal.get_angles(), decimal=4
)
np.testing.assert_array_almost_equal(
    self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
)
print(f"{self.cal.get_pos()}")
print(f"{self.cal.get_angles()}")
print(f"{self.cal.added_par}")

self.orient_par.ccflag=1
self.orient_par.xhflag=0
self.orient_par.yhflag=0
self.orient_par.k1flag=0
self.orient_par.k2flag=0
self.orient_par.k3flag=0
self.orient_par.scxflag=0
self.orient_par.sheflag=0
self.orient_par.p1flag=1
self.orient_par.p2flag=1
print(f"Calibrating with the following flags: {self.orient_par}")

_, _, _ = full_calibration(
    self.cal,
    ref_pts,
    targets,
    self.control,
    self.orient_par
    )

np.testing.assert_array_almost_equal(
    self.cal.get_angles(), self.orig_cal.get_angles(), decimal=4
)
np.testing.assert_array_almost_equal(
    self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
)
print(f"{self.cal.get_pos()}")
print(f"{self.cal.get_angles()}")
print(f"{self.cal.added_par}")

def point_position(
    targets: np.ndarray,
    num_cams: int,
    multimed_pars: MultimediaPar,
    cals: List[Calibration],
) -> Tuple[float, np.ndarray]:
    """
    Calculate an average 3D position implied by the rays.

    sent toward it from cameras through the image projections of the point.

    Arguments:
    ---------
    targets - for each camera, the 2D metric, flat, centred coordinates
        of the identified point projection.
    multimed_pars - multimedia parameters struct for ray tracing through
        several layers.
    cals - each camera's calibration object.

    Returns
    -------
    A tuple containing the ray convergence measure (an average of skew ray distance across all ray pairs)
    and the average 3D position vector.
    """
    # loop counters
    num_used_pairs = 0
    dtot = 0.0
    point_tot = np.array([0.0, 0.0, 0.0])

    vertices = np.zeros((num_cams, 3))
    directs = np.zeros((num_cams, 3))
    point = np.zeros(3)

    # Shoot rays from all cameras.
    for cam in range(num_cams):
        if targets[cam][0] != COORD_UNUSED:
            vertices[cam], directs[cam] = ray_tracing(
                targets[cam][0], targets[cam][1], cals[cam], multimed_pars
            )

    # Check intersection distance for each pair of rays and find position
    for cam in range(num_cams):
        if targets[cam][0] == COORD_UNUSED:
            continue

        for pair in range(cam + 1, num_cams):
            if targets[pair][0] == COORD_UNUSED:
                continue

            num_used_pairs += 1
            tmp, point = skew_midpoint(
                vertices[cam], directs[cam], vertices[pair], directs[pair]
            )
            dtot += tmp
            point_tot += point

    res = point_tot / num_used_pairs
    dtot /= num_used_pairs

    return float(dtot), res.astype(float)

In [ ]:
for i in range(1,6):
    df = pd.read_csv(f'./reconstruction/plane{i}.123456789',skiprows=1,sep='\s+',header=None)
    df.columns = ['id','x','y','z','i0','i1','i2','i3']
    # df.head()
    ground = pd.read_csv(f'./calibration/plane_{i}.csv',header=None)
    ground.columns = ['id','x','y','z']
    # ground.head()


    sc = px.scatter_3d(df, x='x', y='y', z='z',color='id')
    fig2 = go.Figure(data=[go.Scatter3d(x=ground.x, y=ground.y, z=ground.z,
                                    mode='markers')])
    fig2.add_traces(sc.data)
    fig2.show()
